# Saved Snippets
This script holds random snippets of code that I may need again, but are cluttering the main scripts.

## Custom PLSRW

In [ ]:
from scipy.linalg import pinv2
from sklearn.metrics import r2_score

In [ ]:
def _center_scale_xy(X, Y, scale=True):
    """ Center X, Y and scale if the scale parameter==True

    Returns
    -------
        X, Y, x_mean, y_mean, x_std, y_std
    """
    # center
    x_mean = X.mean(axis=0)
    X -= x_mean
    y_mean = Y.mean(axis=0)
    Y -= y_mean
    # scale
    if scale:
        x_std = X.std(axis=0, ddof=1)
        x_std[x_std == 0.0] = 1.0
        X /= x_std
        y_std = Y.std(axis=0, ddof=1)
        y_std[y_std == 0.0] = 1.0
        Y /= y_std
    else:
        x_std = np.ones(X.shape[1])
        y_std = np.ones(Y.shape[1])
    return X, Y, x_mean, y_mean, x_std, y_std

class PLSRW():
    
    def __init__(self, n_components=2, scale=True, reg=0.01):
        self.n_components=n_components
        self.scale = scale
        self.reg = reg
    
    def _calc_dist(self, X, Y):
        dist = []
        
        for feature in range(X.shape[1]):
            feature_dist = np.linalg.norm(Y - X[:, feature])
            dist.append(feature_dist)
        
        return np.array(dist)
    
    def fit(self, X, Y):
        Y = Y.astype('float64')
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)
        
        n = X.shape[0]
        p = X.shape[1]
        q = Y.shape[1]
        
        n_components = self.n_components
        reg = self.reg
        eps = np.finfo(X.dtype).eps
        Y_eps = np.finfo(Y.dtype).eps
        
        self.x_weights_ = np.zeros((p, n_components))  # U
        self._x_scores = np.zeros((n, n_components))  # Xi
        self.x_loadings_ = np.zeros((p, n_components))  # Gamma
        self.y_loadings_ = np.zeros((q, n_components))  # Delta
        
        # Scale (in place)
        Xk, Yk, self._x_mean, self._y_mean, self._x_std, self._y_std = (
            _center_scale_xy(X, Y, self.scale))
        Yk_mask = np.all(np.abs(Yk) < 10 * Y_eps, axis=0)
        Yk[:, Yk_mask] = 0.0
        
        for k in range(n_components):
            # Compute the regularization matrix
            d = self._calc_dist(Xk, Yk)
            D = np.diag(d)
            print(reg * (D.T @ D))
            print((Xk.T @ Xk))
            
            # Compute the PLSRW weight
            w_inter = pinv2(
                ((Xk.T @ Xk) + (reg * (D.T @ D))), check_finite=False)
            x_weights = (w_inter @ Xk.T) @ Yk
            print("x_weights:", x_weights.shape)
            
            # Normalize weight
            x_weights /= np.sqrt(x_weights.T @ x_weights) + eps
            
            # Calculate the corresponding scores and loadings
            x_scores = Xk @ x_weights
            x_loadings = (Xk.T @ x_scores) / (x_scores.T @ x_scores)
            y_loadings = (Yk.T @ x_scores) / (x_scores.T @ x_scores)
            
            # Deflate X and Y
            Xk -= np.outer(x_scores, x_loadings)
            Yk -= np.outer(x_scores, y_loadings)
            print("Xk:", Xk.shape, "Yk:", Yk.shape)
            
            self.x_weights_[:, k] = x_weights[:, 0]
            self._x_scores[:, k] = x_scores[:, 0]
            self.x_loadings_[:, k] = x_loadings[:, 0]
            self.y_loadings_[:, k] = y_loadings[:, 0]

        # Compute transformation matrices
        self.x_rotations_ = np.dot(
            self.x_weights_,
            pinv2(np.dot(self.x_loadings_.T, self.x_weights_),
                  check_finite=False))
        
        self.coef_ = np.dot(self.x_rotations_, self.y_loadings_.T)
        self.coef_ = self.coef_ * self._y_std
        print("coef:", self.coef_.shape)
        return self
    
    def predict(self, X):
        return X @ self.coef_
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

In [ ]:
plsrw = PLSRW(n_components=2, reg=0)
plsrw.fit(X, y)
print(plsrw.score(X, y))

## Display PLS plots while training

In [ ]:
train_scores = []
test_scores = []
# fig, axs = plt.subplots(3, 3, figsize=(15, 15))
# x_fig, y_fig = 0, 0

for i in range(0, 1000):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3)
    
    pls = PLSRegression(n_components=num_component)
    pls.fit(X_train, y_train)

    y_train_pred = pls.predict(X_train)
    y_test_pred = pls.predict(X_test)
    
    train_scores.append(r2_score(y_train, y_train_pred))
    test_scores.append(r2_score(y_test, y_test_pred))
    
#     if (i % 10 == 0):
#         axs[y_fig, x_fig].scatter(y_test, y_test_pred, alpha=0.3)
#         m, b, r, p, std_err = stats.linregress(y_test, y_test_pred[:,0])
#         axs[y_fig, x_fig].plot(y_test, (m * y_test) + b, alpha=0.3)
#         print(x_fig, y_fig, f"r:{r:.2f}", f"r^2:{test_scores[-1]:.2f}")
#         x_fig += 1
#         if (x_fig % 3 == 0):
#             x_fig = 0
#             y_fig += 1

avg_train_score = np.mean(train_scores)
avg_test_score = np.mean(test_scores)

print("Train r^2:", avg_train_score)
print("Test r^2:", avg_test_score)
# plt.show()

## Display Yeo FC

In [ ]:
# To display each class of connections (within and between)
plt.figure(figsize=(10, 10))
plt.imshow(subject_fc)
a = np.zeros((11, 11))
a[fpn_indices] = subject_fc[fpn_indices]
plt.imshow(a)
b = np.zeros((8, 8))
b[np.triu_indices(8, k=1)] = subject_fc[dmn_indices]
plt.imshow(b)
plt.imshow(subject_fc[:11, 11:])

## Plot Brain Images

In [ ]:
# For plotting other functional images
img = image.load_img(f'{subject_path}/wr{subject}_task-movieDM_bold_0375.nii')
concat_img = image.concat_imgs(subject_niftis)
img = image.index_img(concat_img, 0)
img = image.mean_img(concat_img)

## Feature Selection (Poor Performance)

### Variance Threshold

In [ ]:
sel = VarianceThreshold(0.055)
X = sel.fit_transform(X)
print("X shape:", X.shape)

### Select k strongest connections

In [ ]:
def score_fc(X, y):
    # Take the strongest correlations regardless of sign
    sum_fc = np.absolute(np.sum(X, axis=0))
    return sum_fc

X = SelectKBest(score_fc, k=3400).fit_transform(X, y)
print("X shape:", X.shape)